In [1]:
data = [
    ("love this product",1),
    ("This in the worst thing eveer",0),
    ("im so happy with this purchase",1),
    ("this in not what i expected ",0),
    ("amaziing",1),
    ("Horrible",0),
    ("never buying this again",0),
    ("best thing i got",1),
    ("i regret buying this",0),
    ("great quality",1)
]

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader,Dataset
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import numpy as np

In [3]:
from collections import Counter
def tokenize_sentence(sentence):
    return sentence.lower().split()

vocab = Counter()
for sentence , label in data:
    tokens = tokenize_sentence(sentence)
    vocab.update(tokens)

word2index = {word : idx for idx ,(word,_) in enumerate(vocab.items(),1)}
word2index["<PAD>"] = len(word2index)
vocab_size = len(word2index)

def encode_sentence(sentence,word2index,max_len=10):
    tokens = tokenize_sentence(sentence)
    encoded = [word2index.get(token,0) for token in tokens]
    encoded = encoded[:max_len]+[0]*(max_len-len(encoded))
    return encoded

x = [encode_sentence(sentence,word2index) for sentence , label in data]
y = [label for sentence , label in data]

x_train , x_test, y_train , y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [4]:
class SentenceDataset(Dataset):
    def __init__(self,x,y):
        self.x = torch.LongTensor(x)
        self.y = torch.LongTensor(y)

    def __getitem__(self,idx):
        return self.x[idx],self.y[idx]

    def __len__(self):
        return len(self.y)

train_dataset = SentenceDataset(x_train,y_train)
test_dataset = SentenceDataset(x_test,y_test)


train_loader = DataLoader(train_dataset,batch_size=2,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=2,shuffle=False)


In [5]:
class SelfAttention(nn.Module):
    def __init__(self,embed_dim,heads):
        super(SelfAttention,self).__init__()
        self.query= nn.Linear(embed_dim,embed_dim)
        self.key = nn.Linear(embed_dim,embed_dim)
        self.value = nn.Linear(embed_dim,embed_dim)
        self.scale = embed_dim **0.5

    def forward(self,x):
        Q = self.query(x)
        K = self.key(x)
        V = self.value(x)

        attn_wieghts = F.softmax(torch.matmul(Q,K.T(-2,1))/self.scale,dim=1)
        output = torch.matmul(attn_wieghts,V)
        return output , attn_wieghts

In [6]:
class SentenceClassifier(nn.Module):
    def __init__(self,vocab_size,embed_dim,num_classes):
        super(SentenceClassifier,self).__init__()
        self.embedding = nn.Embedding(vocab_size,embed_dim)
        self.attention = SelfAttention(embed_dim,heads=2)
        self.fc = nn.Linear(embed_dim,num_classes)


    def forward(self,x):
        embedded = self.embedding(x)
        attention_output , attn_wieghgts = self.attention(embedded)
        pooled_output = attention_output.mean(dim=1)
        output = self.fc(pooled_output)

        return output , attn_wieghgts

embed_dim = 16
# hidden_dim = 128
num_classes = 2
model = SentenceClassifier(vocab_size,embed_dim,num_classes)

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)

#training loop
num_epochs = 40
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for x_batch,y_batch in train_loader:
        optimizer.zero_grad()